   # KNN met handmatige labeling

voor dit algoritme zijn in enkele pointclouds de vogels handmatig gescheiden van de regen. Deze data is gebruikt als labeling voor het k nearest neighbors algoritme.

Ten eerste zijn van elk punt de 50 dichtsbijzijnde punten bepaald en de variantie in DBZH en het gemiddelde van de DBZH bepaald. Deze stap neemt veel tijd in beslag, maar zorgt voor minder ruis in het eindresultaat. Je zou dit ook kunnen laten voor snellere maar minder precieze resultaten. 

Er wordt vervolgens een array gemaakt met van elk punt zijn DBZH waarde, zijn Z waarde, zijn variantie in DBZH tussen de nabije punten, en het gemiddelde van DBZH in deze punten. Er is geprobeerd om ook de variantie op de VRAD mee te nemen in de resultaten, maar dit leverde geen betere resultaten op. Hier wordt het kNN algoritme op toegepast.

resultaten:
- vrij goed
- soms wordt aan de randjes van wolken nog wat als vogels geclassificeerd
- soms bij overlap worden vogels als wolk geclassificeerd

In [60]:
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
import numpy as np

""" load file maakt van een door cloudcompare geoutputte text file twee np arrays
    De eerste met de xyz waarden van een punt, de tweede met de dbzh waarde """
def load_file(filename):
    originali = np.genfromtxt(filename, delimiter=' ')
    original = originali[1:]
    xyz = original[:, [0, 1, 2]]
    DBZH = original[:, 3]
    return [xyz, DBZH]

""" get_nearest_neigbours vindt voor elk punt in een array de n_neighbours dichtst bijzijnde punten
    De default waarde staat op 100 neighbors """
def get_nearest_neighbours(array, n_neighbors=100):
    nbrs = NearestNeighbors(n_neighbors, algorithm='ball_tree').fit(array)
    distances, indices = nbrs.kneighbors(array)
    return indices

""" get_variances vindt de variantie in locatie en intensiteit binnen een punt en zijn buren.
    er wordt een array gereturned met de DBZH waarden en varianties voor elk punt """
def get_variances(nearest_indices, xyz, DBZH, classification = None):
    if classification:
        columns = 5
    else:
        columns = 4
    db_means = []
    db_vars = []
    for points in nearest_indices:
        db = []
        v = []
        for i in points:
            db.append(DBZH[i])
        db_means.append(np.mean(db))
        db_vars.append(np.var(db))
    zeros = np.zeros((len(DBZH), columns))
    zeros[:, 0] = DBZH
    zeros[:, 1] = xyz[:, 2]
    zeros[:, 2] = db_vars 
    zeros[:, 3] = db_means
    if classification:
        zeros[:, 4] = classification
    return zeros

""" make data neemt een door cloudcompare gemaakte textfile en maakt hier een geschikte numpy
    array van die als training of test data kan worden gebruikt."""
def make_data(filename, class_number):
    file = load_file(filename)
    xyz = file[0]
    DBZH = file[1]
    knn = get_nearest_neighbours(xyz)
    return get_variances(knn, xyz, DBZH, class_number)

""" returned een array met de xyz waarden van een door numpy gegenereerde textfile """
def get_xyz(filename):
    file = load_file(filename)
    return file[0]


In [72]:
birds1 = make_data('labeled/birds/birds1.txt', 1)
birds2 = make_data('labeled/birds/birds2.txt', 1)
birds3 = make_data('labeled/birds/birds3.txt', 1)
birds5 = make_data('labeled/birds/birds5.txt', 1)
birds4 = make_data('labeled/birds/birds4.txt', 1)

clouds1 = make_data('labeled/clouds/clouds1.txt', 2)
clouds2 = make_data('labeled/clouds/clouds2.txt', 2)
clouds3 = make_data('labeled/clouds/clouds3.txt', 2)
clouds4 = make_data('labeled/clouds/clouds4.txt', 2)
clouds5 = make_data('labeled/clouds/clouds5.txt', 2)
clouds6 = make_data('labeled/clouds/clouds6.txt', 2)
clouds7 = make_data('labeled/clouds/clouds7.txt', 2)
clouds8 = make_data('labeled/clouds/clouds8.txt', 2)

total = np.concatenate((birds1, birds2, birds3, birds5, birds4, clouds1, clouds2, clouds3, clouds4, clouds5, clouds6, clouds7, clouds8), axis = 0)

In [74]:
def classify_knn(test_file, training_array, output_file_name, N):
    test_array = make_data(test_file, None)
    neigh = KNeighborsClassifier(n_neighbors=N)
    neigh.fit(training_array[:,[0, 1, 2, 3]], training_array[:, 4]) 
    output_labels = neigh.predict(test_array[:,[0, 1, 2, 3]])
    output = np.append(get_xyz(test_file), np.reshape(output_labels,(-1,1)), axis=1)
    np.savetxt(output_file_name, output, delimiter=',')

classify_knn('labeled/test/test2.txt', total, 'output_csv/knn26.csv', 100)

In [71]:
def remove_vrad(filename, output_name):
    originali = np.genfromtxt(filename, delimiter=' ')
    original = originali[1:]
    np.savetxt(output_name, original[:, [0,1,2,3]], delimiter=' ')
    
remove_vrad('labeled/birds/birdsV2.txt', 'labeled/birds/birds5.txt')
remove_vrad('labeled/birds/birdsV1.txt', 'labeled/birds/birds4.txt')
remove_vrad('labeled/clouds/cloudsV1.txt', 'labeled/clouds/clouds4.txt')
remove_vrad('labeled/clouds/cloudsV2.txt', 'labeled/clouds/clouds5.txt')
remove_vrad('labeled/clouds/cloudsV3.txt', 'labeled/clouds/clouds6.txt')
remove_vrad('labeled/clouds/cloudsV4.txt', 'labeled/clouds/clouds7.txt')
remove_vrad('labeled/clouds/cloudsV5.txt', 'labeled/clouds/clouds8.txt')